# Graph Analysis
### Greedy best-first search
**Greedy best-first search** is an **informed** search algorithm that expands the node that appears to be closest to the goal based solely on the heuristic function. Thus, the evaluation function $f(n)$ is:
- $f(n)=h(n)$, where $h(n)$ is a heuristic estimate to goal - solely greedy guidance

<hr>

Some key points about GBFS for a graph $G=(V,E)$:
- **Time Complexity:** O((|V|+|E|) log |V|) but with a good heuristic it is faster
- **Space complexity:** O(|V|)
- **Optimality:** It is not Guaranteed
- **Completeness:** Always finds a solution if one exists in finite graphs

**Reminder:** **Completeness** in search algorithms refers to whether the algorithm is guaranteed to find a solution if one exists.
<hr>

**Hint:** Despite not being optimal, GBFS is used when:
- Speed matters more than optimality (e.g., real-time games)
  - Common in game AI for quick path approximations
- Memory is limited (simpler than A*)
- Heuristic is very accurate (e.g., GPS with clear line-of-sight)
- As a component in hybrid algorithms (e.g., beam search)  
     
<hr>

In the following, we implement greedy best-first search (GBFS) with a min-heap as a priority queue (similar to the Dijkstra's algorithm and A* algorithm). Then, we use if for a simple grid graph to find the possibly shortest path from a start node to a goal node. 

https://github.com/ostad-ai/Graph-Analysis
<br>Explanation in English :https://www.pinterest.com/HamedShahHosseini/graph-analysis/

In [1]:
# import required module
import heapq

In [2]:
# Define Greedy BFS (GBFS) implementation
def greedy_best_first_search(graph, start, goal, heuristic):
    '''graph: an adjacency list '''
    '''start: is the start node to search from'''
    '''goal: is the end node to reach'''
    '''heuristic: estimates distance to goal from current node'''
    open_set = [] # priority queue
    heapq.heappush(open_set, (heuristic(start, goal), start))
    came_from = {}     # to reconstruct the path
    visited = set()    # visited set of nodes
    nodes_expanded = 0
    
    came_from[start] = None # parent of start
    
    while open_set: # while priority queue is not empty
        current_h, current = heapq.heappop(open_set)
        nodes_expanded += 1
        
        if current == goal:
            path = reconstruct_path(came_from, current)
            return path, nodes_expanded
        
        # Skip if already expanded (visited set is sufficient)
        if current in visited:
            continue
            
        visited.add(current)
        
        # Expand neighbors - ONLY check visited set
        for neighbor in graph.get(current, []):
            if neighbor not in visited:
                # Update came_from when finding a new way to reach neighbor
                came_from[neighbor] = current
                h_value = heuristic(neighbor, goal)
                heapq.heappush(open_set, (h_value, neighbor))
    
    return None, nodes_expanded

In [3]:
def reconstruct_path(came_from, current):
    """Reconstruct path from start to current node"""
    path = [current]
    while came_from[current] is not None:
        current = came_from[current]
        path.append(current)
    return path[::-1]  # Reverse to get start->goal

# Common heuristics
def manhattan_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def euclidean_distance(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

In [5]:
"""Greedy BFS on a grid showing suboptimal behavior"""
# Create a grid 
grid = [
    [0, 0, 0, 0, 0, 1, 0],
    [0, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0]
]

# Build graph
grid_graph = {}
for x in range(len(grid)):
    for y in range(len(grid[0])):
        if grid[x][y] == 1:
            continue
        neighbors = []
        for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
            nex, ney = x + dx, y + dy
            if (0 <= nex < len(grid) and 0 <= ney < len(grid[0]) 
                and grid[nex][ney] == 0):
                neighbors.append((nex, ney))
        grid_graph[(x, y)] = neighbors

start = (0, 0)
goal = (4, 6)

path, nodes_expanded = greedy_best_first_search(
    grid_graph, start, goal, manhattan_distance
)

print("\nGrid Example - Demonstrating Suboptimality:")
print("=" * 50)
print(f"Start: {start}, Goal: {goal}")
print(f"Manhattan distance from start: {manhattan_distance(start, goal)}")
print(f"Path length: {len(path) if path else 'No path'}")
print(f"Nodes expanded: {nodes_expanded}")

# Visualize
if path:
    print("\nGrid Visualization:")
    for x in range(len(grid)):
        row = []
        for y in range(len(grid[0])):
            if (x, y) == start:
                row.append('S')
            elif (x, y) == goal:
                row.append('G')
            elif grid[x][y] == 1:
                row.append('█')
            elif (x, y) in path:
                row.append('•')
            else:
                row.append('.')
        print(' '.join(row))


Grid Example - Demonstrating Suboptimality:
Start: (0, 0), Goal: (4, 6)
Manhattan distance from start: 10
Path length: 11
Nodes expanded: 11

Grid Visualization:
S • • • • █ .
. █ █ █ • • •
. . . . . █ •
. █ █ █ █ █ •
. . . . . . G
